In [1]:
import torch
import torch.nn as nn
from torch.quantization import quantize_dynamic
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch.utils.tensorboard import SummaryWriter
import os
from utils.processing import *
from utils.pilot_net_dataset import PilotNetDataset
from utils.pilotnet import PilotNet
from utils.transform_helper import createTransform
import argparse
import json
import numpy as np
from copy import deepcopy
import pandas as pd
import time


# Device Selection (CPU/GPU)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu") # support available only for cpu
cast = torch.Tensor


/home/docker/miniconda3/envs/dlstudio2/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_save_dir = './experiments/optimized_models/trained_models'
qmodel_path = model_save_dir + '/static_quan.ckpt'
quant_model = torch.jit.load(qmodel_path)
# print(quant_model)

In [3]:
# quant_model._weight_bias()
# for p in quant_model.named_children():
#     print(p)

In [4]:
pilotModel = PilotNet([200, 66, 3], 2).to(device)
# pilotModel.load_state_dict(torch.load('experiments/retrain_best/trained_models/pilot_net_model_121.ckpt'))
print(pilotModel)

PilotNet(
  (ln_1): BatchNorm2d(3, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  (cn_1): Conv2d(3, 24, kernel_size=(5, 5), stride=(2, 2))
  (cn_2): Conv2d(24, 36, kernel_size=(5, 5), stride=(2, 2))
  (cn_3): Conv2d(36, 48, kernel_size=(5, 5), stride=(2, 2))
  (cn_4): Conv2d(48, 64, kernel_size=(3, 3), stride=(1, 1))
  (cn_5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc_1): Linear(in_features=1152, out_features=1164, bias=True)
  (fc_2): Linear(in_features=1164, out_features=100, bias=True)
  (fc_3): Linear(in_features=100, out_features=50, bias=True)
  (fc_4): Linear(in_features=50, out_features=10, bias=True)
  (fc_5): Linear(in_features=10, out_features=2, bias=True)
)


In [9]:
# for p in m.children():
#     print(p)

In [5]:
m = deepcopy(pilotModel)
m.eval()
backend = "fbgemm"
"""Insert stubs"""
m = nn.Sequential(torch.quantization.QuantStub(), 
                  *nn.Sequential(*(m.children())), 
                  torch.quantization.DeQuantStub())

"""Prepare"""
m.qconfig = torch.quantization.get_default_qconfig(backend)
torch.quantization.prepare(m, inplace=True)

"""Calibrate
- This example uses random data for convenience. Use representative (validation) data instead.
"""
with torch.inference_mode():
  for _ in range(10):
    x = torch.rand(1,3, 200, 66)
    m(x)
    
"""Convert"""
torch.quantization.convert(m, inplace=True)


/home/docker/miniconda3/envs/dlstudio2/lib/python3.10/site-packages/torch/ao/quantization/observer.py:177: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


Sequential(
  (0): Quantize(scale=tensor([0.0080]), zero_point=tensor([0]), dtype=torch.quint8)
  (1): QuantizedBatchNorm2d(3, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  (2): QuantizedConv2d(3, 24, kernel_size=(5, 5), stride=(2, 2), scale=0.019769079983234406, zero_point=64)
  (3): QuantizedConv2d(24, 36, kernel_size=(5, 5), stride=(2, 2), scale=0.011379468254745007, zero_point=66)
  (4): QuantizedConv2d(36, 48, kernel_size=(5, 5), stride=(2, 2), scale=0.007137960754334927, zero_point=62)
  (5): QuantizedConv2d(48, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.004424086306244135, zero_point=54)
  (6): QuantizedConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.0025282984133809805, zero_point=62)
  (7): Flatten(start_dim=1, end_dim=-1)
  (8): QuantizedLinear(in_features=1152, out_features=1164, scale=0.0019415776478126645, zero_point=61, qscheme=torch.per_channel_affine)
  (9): QuantizedLinear(in_features=1164, out_features=100, scale=0.0010156267089769244

In [13]:
# m.__dict__ 
# print(m[0][[1]].weight().element_size())

for p in m.named_parameters():
    print(p)

('1.weight', Parameter containing:
tensor([1., 1., 1.], requires_grad=True))
('1.bias', Parameter containing:
tensor([0., 0., 0.], requires_grad=True))


In [16]:
# m._modules['1'].ln_1.weight.element_size()

In [17]:
img = torch.rand(1,3,200,66)

m.eval()
with torch.no_grad():
    out = m(img)